In [1]:
using DataFrames, LinearAlgebra, Revise, Serialization

In [41]:
string(@__DIR__) in LOAD_PATH || push!(LOAD_PATH, @__DIR__)
using BLPmodule; const m = BLPmodule;

---
#### Mean utilities

In [52]:
df = deserialize("jls/df.jls");
firm_IDs_long = df.j;
tract_IDs_long = df.t;
X = Matrix(df[!, [:x1, :x2]])
D = Matrix(df[!, [:d, :d2]])
Q = df.q
M = df.M;
nJ = length(unique(firm_IDs_long));
nI = 100;

In [53]:
σ = ones(2)
pars = m.set_Pars(K = 2, nI = nI, δs=ones(nJ,1));
serialize("jls/pars.jls", pars)

---

In [54]:
ec = m.make_Economy(
    firm_IDs_long,
    tract_IDs_long,
    X,
    D,
    Q,
    M,
    nI
    );


Economy with 394 firms and 3110 tracts.


In [55]:
serialize("jls/ec.jls", ec)

---

In [39]:
Threads.nthreads()

16

In [20]:
σ = ones(2);

In [21]:
ec = deserialize("jls/ec.jls");
pars = deserialize("jls/pars.jls");
deltas, q_iter = m.compute_deltas(ec, pars, σ,max_iter=1000, verbose=true);
# serialize("jls/deltas.jls", deltas);

dist = 9.66707447425108e-7, iterations = 276


In [11]:
ec.tracts[1].shares

2×1 Matrix{Float64}:
 0.01928049695256938
 0.025761485944680477

In [13]:
using BenchmarkTools

In [14]:
ec = deserialize("jls/ec.jls");
pars = deserialize("jls/pars.jls");
it = 1
@time m.compute_deltas(ec, pars, σ, max_iter=it, verbose=false);

TypeError: TypeError: in new, expected Vector{Real}, got a value of type Vector{Float64}

In [15]:
ec = deserialize("jls/ec.jls");
pars = deserialize("jls/pars.jls");
it = 10
@time m.compute_deltas(ec, pars, σ, max_iter=it, verbose=false);

TypeError: TypeError: in new, expected Vector{Real}, got a value of type Vector{Float64}

In [16]:
ec = deserialize("jls/ec.jls");
pars = deserialize("jls/pars.jls");
it = 1000
@time m.compute_deltas(ec, pars, σ, max_iter=it, verbose=false);

TypeError: TypeError: in new, expected Vector{Real}, got a value of type Vector{Float64}

In [17]:
Threads.nthreads()

16

In [18]:
# # this was the line that the memory profiler said was creating the vast majority of allocations
# # ...which doesn't make sense because it's not even in the inner loop
# # ...and it's the inner loop that's driving the allocations and times (according to benchmarks)
# @time for t in ec.tracts
#     t.abδ .= t.D * pars.nlcoefs
# end

___
continued in `gmm.ipynb`